In [8]:
import os

config = {}
config["weights_file"] = os.getcwd() + '/model/weight'
config["patch_size"] = (64, 64, 64)  # switch to None to train on the whole image
config["patch_gap"] = 16
config["batch_size"] = 10
config["kfold"] = 5

config["input_shape"] = (1, None, None, None)
config["depth"] = 4 # depth of layers for V/Unet
config["n_base_filters"] = 32
config["pool_size"] = (2, 2, 2)  # pool size for the max pooling operations
config["deconvolution"] = True  # if False, will use upsampling instead of deconvolution

config["patience"] = 10  # learning rate will be reduced after this many epochs if the validation loss is not improving
config["early_stop"] = 10  # training will be stopped after this many epochs without the validation loss improving
config["initial_learning_rate"] = 0.00001
config["learning_rate_drop"] = 0.5  # factor by which the learning rate will be reduced
config["n_epochs"] = 1

In [9]:
from model.data import *
from model.model import *

d = Data()
d.load_data()

# prepare data for training
train_num, valid_num = d.prekfold(config["patch_size"], config["patch_gap"], config["batch_size"], config["kfold"])

In [10]:

model = unet_model_3d(input_shape=config["input_shape"],
                              pool_size=config["pool_size"],
                              initial_learning_rate=config["initial_learning_rate"],
                              deconvolution=config["deconvolution"],
                              depth=config["depth"],
                              n_base_filters=config["n_base_filters"])

model.load_weights(os.getcwd() + '/model/weight/weights-01-0.00.hdf5')


In [12]:
image = []
target = []
result = []

count = 0
for img, tar in d.valid_generator(0):
    if count == 20:
        break
    image.append(img)
    target.append(tar)
    result.append(model.predict(img))
    count += 1

In [18]:
sel = 10
d.show_image([image[sel], target[sel], result[sel]])

# show_image([result[0], image[0][None, :, :, :, :], target[0][None, :, :, :, :]])


interactive(children=(IntSlider(value=32, description='id', max=63), Output()), _dom_classes=('widget-interact…

0.0


In [19]:
print(np.mean(result))

4.6268564e-05
